In [11]:
using HDF5, EzXML, Glob, MAT

datapath = "/Users/jjc/Documents/SSC/CSF Data/selected_recordings"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

for filename in files
    
# filename = "INF_20200909122333_INF2.hdf5"
# filename = datapath*"/INF_20180829151805_INF2.hdf5"
    println("Processing "*filename*" ...")

fid = h5open(filename, "r")

# Get attributes outside of datasets/groups
recording_end_time = read_attribute(fid, "dataEndTime")
recording_start_time = read_attribute(fid, "dataStartTime")
dur = read_attribute(fid, "duration")
dur = dur[1];
dur = split(dur, " ")
recording_duration = parse(Int64, dur[1])

# Begin handling data in XML string - some files do not have infusion test output
try
    xml_obj = fid["aux/ICM+/icmtests"]
catch
    println("No infusion test data \n")
    continue
end
    
xml_data = read(xml_obj)
xml_string = String(xml_data[1])

data = parsexml(xml_string)
icm_data = root(data)
vars = elements(icm_data)

SingleAnalysis = firstelement(vars[2])
SingleAnalysis = elements(SingleAnalysis)
Variables = elements(SingleAnalysis[1])
Selections = elements(SingleAnalysis[2])
Parameters = elements(SingleAnalysis[3])
Results = elements(SingleAnalysis[4])

inf_rate = parse(Float64, Parameters[1]["Value"])

function parseXML(Parameters, Selections)
    
    selections = Dict{String, String}
    results = Dict{String, Float64}
    
    for i in 1:length(Selections)
        sel = elements(Selections[i])
        sel_name = Selections[i]["Name"]
        sel_st = sel[1]["StartTime"]
        sel_en = sel[1]["EndTime"]  
        selections =  merge!(selections, Dict(sel_name => sel_st*" to "*sel_en))
    end

    for i in 1:length(Results)
        res = elements(Results[i]);
        res_name = Results[i]["Name"]
        name_surrogate = split(res_name, " ");
        res_name = name_surrogate[1];
        res_val = Results[i]["Value"]
        results =  merge!(results, Dict(res_name => parse(Float64, res_val)))
    end
    return results, selections
end

results, selections = parseXML(Parameters, Selections);

# This dereferencing is pain, there has to be a more elegant solution
t_series_ds = fid["summaries/minutes/"]
t_series = t_series_ds[:]
# Pre-allocate
ICP = zeros(1,length(h))
AMP = zeros(1,length(h))
timestamp = zeros(1,length(h));
P0 = zeros(1,length(h));
AMP_P = zeros(1,length(h));
# Dereferncing named tuple...
for i in 1:length(g)
    ICP[i] = h[i].ICP
    AMP[i] = h[i].AMP
    timestamp[i] = h[i].datetime
    P0[i] = h[i].P0
    AMP_P[i] = h[i].AMP_P
end

# OUTPUT:
# results - results and estimated parameters
# selections - start and end times for events marked in infusion test
# recording_start_time - datetime
# recording_end_time - datetime
# recording_duration - seconds
# inf_rate - infusion rate [mL/min]
# TIME SERIES DATA:
# ICP - ICP
# AMP - ICP pulse amplitude
# timestamp - reference timestamps for x-axis
# P0 - reference pressure
# AMP_P - ICP wave amplitude at plateau

    println("Saving to "*filename[1:end-4]*".mat ...\n")
    matwrite(filename[1:end-5]*".mat", Dict(
    "results" => results,
    "selections" => selections,
    "recording_start_time" => recording_start_time[1],
    "recording_end_time" => recording_end_time[1],
    "recording_duration" => recording_duration,
    "inf_rate"=> inf_rate,
    "ICP" => ICP,
    "AMP" => AMP,
    "timestamps" => timestamp,
    "P0" => P0,
    "AMP_P" => AMP_P
))
end

Processing /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20180829151805_INF2.hdf5 ...
No infusion test data 

Processing /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20190620142837_INF2.hdf5 ...
Saving to /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20190620142837_INF2..mat ...

Processing /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20190701113430_INF2.hdf5 ...
Saving to /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20190701113430_INF2..mat ...

Processing /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20190703121752_INF2.hdf5 ...
No infusion test data 

Processing /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20190708150215_INF2.hdf5 ...
Saving to /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20190708150215_INF2..mat ...

Processing /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20191010113919_INF2.hdf5 ...
Saving to /Users/jjc/Documents/SSC/CSF Data/selected_recordings/INF_20